In [ ]:
import os
import csv
from pathlib import Path
import pandas as pd
import numpy as np

from transformers import pipeline

In [ ]:
languages = ['arabic','bulgarian','english','german','italian']

In [ ]:
pipe = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment", tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment", top_k=None)

In [ ]:
def extract_sentiment(text):
    sentiments = pipe(text)[0]
    return {k:v for k,v in [(list(sentiment.values())[0], list(sentiment.values())[1]) for sentiment in sentiments]}

In [4]:
all_data = []

for language in languages:
    print('Processing', language)
    folder = f'data{os.sep}{language}'
    files = os.listdir(folder)
    
    language_test = None
    language_dev = None
    
    for file in files:
        if file.endswith('.tsv'):
            if 'test' in file:
                test = pd.read_csv(f'{folder}{os.sep}{file}', sep='\t', quoting=csv.QUOTE_NONE)
                test[['positive', 'neutral', 'negative']] = test.apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
                test['language'] = language
                test['set_type'] = 'test'
                language_test = test
            elif 'dev' in file and 'test' not in file:
                dev = pd.read_csv(f'{folder}{os.sep}{file}', sep='\t', quoting=csv.QUOTE_NONE)
                dev[['positive', 'neutral', 'negative']] = dev.apply(lambda x: extract_sentiment(x['sentence']), axis=1, result_type='expand')
                dev['language'] = language
                dev['set_type'] = 'dev'
                language_dev = dev
    
    if language_test is not None:
        all_data.append(language_test)
    if language_dev is not None:
        all_data.append(language_dev)

# Combine all data into a single dataframe
combined_df = pd.concat(all_data, ignore_index=True)
print(f"Combined dataset has {len(combined_df)} rows and includes data from {combined_df['language'].nunique()} languages")

Processing arabic


NameError: name 'extract_sentiment' is not defined

In [ ]:
df = pd.read_csv("predictions_to_be_compared.csv")

df.head()

# wrong prediction per model

In [ ]:
wrong_thr = df[df["true_labels"] != df["english-thr"]]
wrong_bert = df[df["true_labels"] != df["english-modern-bert-thr"]]
wrong_sentiment = df[df["true_labels"] != df["english-sentiment-thr"]]

In [ ]:
print(f"english-thr model got {len(wrong_thr)} wrong predictions")
print(f"english-modern-bert-thr model got {len(wrong_bert)} wrong predictions")
print(f"english-sentiment-thr model got {len(wrong_sentiment)} wrong predictions")


# What the sentiment model got right

0 is objective

In [ ]:
new_pd = df[(df["true_labels"] == df["english-sentiment-thr"]) & (df["english-sentiment-thr"] != df["english-thr"])]
new_pd = new_pd.drop(columns=["english-sentiment-thr", "english-thr", "english-modern-bert-thr", "sentence_id"])

print("For correctly identified by sentiment but wrongly identified by english-thr")
print("len: ",len(new_pd))
print()
print("Mean and std of positive sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["positive"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["positive"].std())
print("Mean and std of neutral sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["neutral"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["neutral"].std())
print("Mean and std of negative sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["negative"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["negative"].std())
print("-"*80)
print("Mean and std of positive sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["positive"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["positive"].std())
print("Mean and std of neutral sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["neutral"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["neutral"].std())
print("Mean and std of negative sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["negative"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["negative"].std())

In [ ]:
new_pd = df[(df["true_labels"] != df["english-sentiment-thr"]) & (df["true_labels"] == df["english-thr"])]
new_pd = new_pd.drop(columns=["english-sentiment-thr", "english-thr", "english-modern-bert-thr", "sentence_id"])

print("For correctly identified by english-thr but wrongly identified by sentiment")
print("len: ",len(new_pd))
print()
print("Mean and std of positive sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["positive"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["positive"].std())
print("Mean and std of neutral sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["neutral"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["neutral"].std())
print("Mean and std of negative sentiment for objective: ",new_pd[new_pd["true_labels"] == 0]["negative"].mean(),"-",new_pd[new_pd["true_labels"] == 0]["negative"].std())
print("-"*80)
print("Mean and std of positive sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["positive"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["positive"].std())
print("Mean and std of neutral sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["neutral"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["neutral"].std())
print("Mean and std of negative sentiment for subjective: ",new_pd[new_pd["true_labels"] == 1]["negative"].mean(),"-",new_pd[new_pd["true_labels"] == 1]["negative"].std())

From the results above, you can see that the sentiment model is able to recognize subjectivity better in the context of high negativity sentiment, while for all the other cases the sentiment is almost perfectly balanced (~ 1/3)